# Двоичное представление

## Двоичная система счисления

Целое число может быть задано **двоичным целочисленным литералом** (*binary integer literal*). Например, число 12 может быть задано литералом `-0b1100`.

Целое число может быть представлено в двоичном представлении (в виде) при помощи функции `bin()`. При этом у двоичного представления будет префикс `0b`.

F-string также предоставляет возможность представления целого числа в виде строки, содержащей двоичное представление. 

In [183]:
# Двоичный целочисленный литерал
x =  0b1010  #  10
y = -0b1100  # -12

# Представление в двоичной системе счисления виде строки
bin(x), f"{y:b}", f"{y:08b}"

('0b1010', '-1100', '-0001100')

Заметим, что производится представление именно в двоичной системе, а не кодирование дополнительном коде. В ячейках памяти целые числа хранятся в дополнительном коде, поэтому нужно всегда учитывать, что последовательности нулей и единиц, возвращаемые функцией `bin()` и форматированием F-string – это представление чисел именно в двоичной системе счисления.

### Методы `bit_length()` и `bit_count()`

У объектов `int` имеются методы `bit_length()` и `bit_count()`. 

Метод `bit_length()` возвращает количество бит, нужных для представления абсолютного значения числа в двоичном виде (без знака и ведущих нулей).

Метод `bit_count()` возвращает установленных битов (единиц) в двоичном представлении абсолютного значения числа.

In [184]:
x, y = 13, 33

f"{x: b}, {y: b}"

' 1101,  100001'

In [185]:
x.bit_length(), y.bit_length()

(4, 6)

In [186]:
x.bit_count(), y.bit_count()

(3, 2)

## Дополнительный код

**Дополнительный код** (*two’s complement*) используется для представления целых отрицательных чисел. Положительные числа совпадают с прямым кодированием, а вот отрицательные числа представляют собой инверсию положительных, с последующим прибавлением единицы. Например число $42$ будет кодировано в `00101010`, а число $-42$ – в `11010110`. Таким образом старший бит также является знаковым. Если число представляется одним байтом, то при такой кодировке

- Имеется только один ноль: `00000000`. 
- $-1$ кодируется сплошными единицами: `11111111`.
- Минимальное число $-128$ кодируется `10000000`.
- Максимальное число $127$ кодируется `01111111`.

Инверсия этого числа используется для кодирования предельного отрицательного числа ($-128$). Диапазон возможных значений $[-128, 127]$.


### Функция для отображения в дополнительном коде

Напишем функцию, которая возвращает строку, которая содержит дополнительный код числа, ограниченный слева. Для этого сформируем положительное число, двоичное представление которого соответствует дополнительному коду заданного числа:
- Выражение `1 << bits` возвращает $2^{\mathrm{bits}}$. При `bits=8` это будет число $256$ и его двоичное представление  `...0001_0000_0000`.
- Выражение `(1 << bits) - 1` возвращает значение $255$, и его двоичное представление – `...0000_1111_1111`. Таким образом мы сделали маску, которая будет обрезать слева все что левее восьмого бита при операции побитовой дизъюнкции `&`.
- В выражении `twos_zero & n` производится побитовая дизъюнкция. Оператор `&` работает непосредственно с дополнительным кодом чисел. Дополнительный код положительного числа $255$ совпадает с его двоичным представлением. Дополнительный код отрицательного числа имеет бесконечный префикс из единиц. Например, число $-12$ в дополнительном коде – `...1111_1111_0100`. Дизъюнкция с `mask` обнулит все, что левее восьмого бита, остальное останется без изменений и получится `...0000_1111_0100`. 

Фактически `...0000_1111_0100` – это положительное число в дополнительном коде, равное 244. Но здесь мы получаем такое число для того, чтобы сформировать строку, содержащую дополнительный код, обрезанный слева.

In [187]:
def to_twos(n:int, bits: int=8) -> str:
    mask = (1 << bits) - 1   # при bits=8 это будет ...000011111111
    twos_n = mask & n        # mask обнулит все, что левее 8-го бита
    return f"{twos_n:0{bits}b}"

to_twos(-1), to_twos(1, bits=16)

('11111111', '0000000000000001')

# Сдвиг

Сдвиг двоичного представления числа приводит эквивалентен умножению/делению на степени двойки.

Влево на $k$ битов (младшие нули добавляются справа):

$$
x \ll k \;=\; x · 2^k
$$ 

Вправо на $k$ битов (младшие разряды удаляются):
$$
x \gg k \;=\; \left\lfloor \frac{x}{2^k} \right\rfloor
$$

Сдвиг вправо приводит к делению числа на $2^k$ с округлением вниз.

In [188]:
x = 25
x >> 3, x // 2**3

(3, 3)

In [189]:
x = -25
x << 2, x * 2**2

(-100, -100)

# Инверсия

Инверсия целого числа $x$, закодированного в дополнительном коде, эквивалентна операции $-x - 1$. Повторное применение оператора `~`, конечно же, возвращает исходное число.

In [190]:
x = 25
~x, ~~x

(-26, 25)

In [191]:
to_twos(x), to_twos(~x), to_twos(~~x)   # дополнительный код

('00011001', '11100110', '00011001')

# AND, OR, XOR

Оператор конъюнкции `|` и оператор дизъюнкции `&` производят битовые операции над разрядами чисел, представленных дополнительным кодом.  Например, число $-12$, который имеет двоичное представление `1100`, в дополнительном коде с 8 разрядами будет `11110100` – т.е. производится инверсия двоичного представления положительного числа прибавляется единица. 

Исключающее ИЛИ производится оператором `^`. 

В Python побитовые операторы работают так, как будто числа представлены в дополнительном коде с бесконочным количеством бит. У отрицательных чисел – «бесконечные» ведущие `1`, а у положительных – `0`.

In [192]:
x =      10  #  11
y = -0b1101  # -13

print(x, y)             # десятичное представление

10 -13


Для демонстрации того, что происходит на уровне дополнительного кода, воспользуемся функцией `to_twos()`.

Дизъюнкция битов (AND)

In [193]:
x_twos, y_twos = to_twos(x), to_twos(y)  # дополнительный код

print(f"{x_twos}, decimal: {x}")
print(f"{y_twos}, decimal: {y}")

disjunct = x & y
disjunct_twos = to_twos(disjunct)

print(f"{disjunct_twos}, decimal: {disjunct}")


00001010, decimal: 10
11110011, decimal: -13
00000010, decimal: 2


Конъюнкция битов (OR)

In [194]:
print(f"{x_twos}, decimal: {x}")
print(f"{y_twos}, decimal: {y}")

conjunct = x | y
conjunct_twos = to_twos(conjunct)

print(f"{conjunct_twos}, decimal: {conjunct}")

00001010, decimal: 10
11110011, decimal: -13
11111011, decimal: -5


Исключающее ИЛИ (XOR)

In [195]:
print(f"{x_twos}, decimal: {x}")
print(f"{y_twos}, decimal: {y}")

xor = x ^ y
xor_twos = to_twos(xor)

print(f"{xor_twos}, decimal: {xor}")

00001010, decimal: 10
11110011, decimal: -13
11111001, decimal: -7
